# VAE for Intact:
---

## Architecture

Input Layer
- **Shape:** `(number_of_features,)`

Encoder
- **First Dense Layer:**
  - Units: `encoder_units` (optimized by Bayesian search)
  - Activation: ReLU
- **Second Dense Layer:**
  - Units: `encoder_units // 2`
  - Activation: ReLU
- **Latent Space Layers:**
  - **z_mean:**
    - Dense layer with `latent_dim` units (optimized by Bayesian search)
    - No activation
  - **z_log_var:**
    - Dense layer with `latent_dim` units
    - No activation
- **Sampling Layer:**
  - Lambda layer for the reparameterization trick
  - Combines `z_mean` and `z_log_var` to sample `z`

Latent Space
- **Dimensions:** `latent_dim` (optimized by Bayesian search)

Decoder
- **Latent Input Layer:**
  - Takes latent variables `z` as input
- **First Dense Layer:**
  - Units: `encoder_units // 2`
  - Activation: ReLU
- **Second Dense Layer:**
  - Units: `encoder_units`
  - Activation: ReLU
- **Output Layer:**
  - Dense layer with the same number of units as input features
  - Activation: Sigmoid

VAE Model
- Combines encoder and decoder
- Loss Function: VAE loss function including:
  - Reconstruction loss (Mean Squared Error)
  - KL divergence loss

Training
- Optimizer: Adam
- Cross-validation: K-Fold (5 splits) to ensure robustness

Hyperparameter Optimization
- Optimized using `optuna` to find the best hyperparameters:
  - `latent_dim`
  - `encoder_units`

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
import optuna
from tensorflow.python.client import device_lib

want feature: "WB16A3C"  
model with file: "C:/Users/dvirz/Desktop/Perimeter/Intact/datasets/J334309/J334309.xlsx"

In [ ]:
file_path = 'C:/Users/dvirz/Desktop/Perimeter/Intact/datasets/J334309/J334309.csv'
data = pd.read_csv(file_path)

# target column
X = data['WB16A3C'].values
features = data.drop(columns=['WB16A3C'])
# standerdizing the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(device_lib.list_local_devices())

In [ ]:
def create_vae_model(trial):
    latent_dim = trial.suggest_int('latent_dim', 2, 10)
    encoder_units = trial.suggest_categorical('encoder_units', [64, 128, 256])
    decoder_units = encoder_units  # for simplicity
    
    # encoder
    inputs = tf.keras.Input(shape=(features_scaled.shape[1],))
    h = layers.Dense(encoder_units, activation='relu')(inputs)
    h = layers.Dense(encoder_units // 2, activation='relu')(h)
    z_mean = layers.Dense(latent_dim)(h)
    z_log_var = layers.Dense(latent_dim)(h)

    def sampling(args):
        z_mean, z_log_var = args
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

    z = layers.Lambda(sampling)([z_mean, z_log_var])

    encoder = tf.keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # decoder
    latent_inputs = tf.keras.Input(shape=(latent_dim,))
    h = layers.Dense(encoder_units // 2, activation='relu')(latent_inputs)
    h = layers.Dense(encoder_units, activation='relu')(h)
    outputs = layers.Dense(features_scaled.shape[1], activation='sigmoid')(h)

    decoder = tf.keras.Model(latent_inputs, outputs, name='decoder')

    # VAE Model
    outputs = decoder(encoder(inputs)[2])
    vae = tf.keras.Model(inputs, outputs, name='vae')

    # loss function
    reconstruction_loss = tf.keras.losses.mse(inputs, outputs) * features_scaled.shape[1]
    kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')

    return vae

# obj for optuna train-test-validation (K-Fold);
# validation done in the hyperparameter optimization stage via optuna
def objective(trial):
    vae = create_vae_model(trial)
    
    # K-Fold Cross Validation
    kf = KFold(n_splits=5)
    val_loss = []
    
    for train_index, val_index in kf.split(features_scaled):
        X_train, X_val = features_scaled[train_index], features_scaled[val_index]
        vae.fit(X_train, X_train, epochs=50, batch_size=32, verbose=0)
        loss = vae.evaluate(X_val, X_val, verbose=0)
        val_loss.append(loss)
    
    return np.mean(val_loss)

In [ ]:
# optuna study and optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# best hyperparameters
print('Best hyperparameters: ', study.best_params)

# training final model with best hyperparameters
best_params = study.best_params
final_vae = create_vae_model(optuna.trial.FixedTrial(best_params))
final_vae.fit(features_scaled, features_scaled, epochs=50, batch_size=32)

# extracting features
encoder_model = tf.keras.Model(final_vae.input, final_vae.get_layer('encoder').output[0])
extracted_features = encoder_model.predict(features_scaled)

# saving extracted features
extracted_features_df = pd.DataFrame(extracted_features, columns=[f'feature_{i}' for i in range(best_params['latent_dim'])])
extracted_features_df['WB16A3C'] = X

extracted_features_df.to_csv('extracted_features.csv', index=False)

In [ ]:
extracted_features_df
#predict new values with the model
new_data = pd.read_csv('new_data.csv')
new_data_scaled = scaler.transform(new_data)
new_features = encoder_model.predict(new_data_scaled)
new_features